In [ ]:
import h5py
from glob import glob
import random
import csv

import numpy as np
import cv2

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

from tools.IPDL import inference

In [ ]:
model = inference.createModel("tools/IPDL/model_weights.keras")

label_file = "IN/labels.csv"

h5py_files = glob('IN/H5PY/dataset_h5py/*.h5py')

In [ ]:
treatments = {}
with open(label_file, 'r') as f:
    file = csv.reader(f, delimiter=',')
    for line in file:
        treatments[line[0]] = {}
        for i in range(1, 33):
            treatments[line[0]][i] = bool(int(line[i][-1]))

In [ ]:
kernel = np.ones((5,5))

In [ ]:
for p in h5py_files:
    with h5py.File(p) as f:
        img = cv2.resize(np.array(f['X'])[...,0], (1024, 512), cv2.INTER_NEAREST)/255
        name = os.path.splitext(os.path.basename(p))[0]
        
        tooth_t = [t for t in treatments[name] if treatments[name][t]]
        tooth_inpaint = []

        inpaint = inference.infer(model, img)
        
        # plt.imsave("OUT/original/" + name + ".png", img, cmap='gray')
        # plt.imsave("OUT/totalinpaint/" + name + ".png", inpaint, cmap='gray')
        # plt.imsave("OUT/selective_IP/" + name + ".png", (inpaint if len(tooth_t) else img), cmap='gray')

        for i in tooth_t:
            segm = cv2.resize(np.array(f['y'])[...,i], (1024, 512), cv2.INTER_NEAREST)
            segm = cv2.dilate(segm, kernel, iterations=1)
            tooth_inpaint.append(np.multiply(segm, inpaint))

        smartinpaint = img
        for tooth in tooth_inpaint:
            smartinpaint = np.where(tooth, tooth, smartinpaint) 
            
        for i in range(len(tooth_inpaint)+1):
            inpaint_i = img
            selected_treatments = random.sample(tooth_inpaint, i)
            for tooth in selected_treatments:
                inpaint_i = np.where(tooth, tooth, inpaint_i)
            plt.imsave("OUT/data_aug_IP/" + name + "_" + str(i) + ".png", inpaint_i, cmap='gray')

            
                
        plt.imsave("OUT/data_aug_total_IP/" + name + ".png", smartinpaint, cmap='gray')